In [25]:
import pandas as pd
import string
from nltk.translate.bleu_score import sentence_bleu

In [2]:
train_df = pd.read_csv("generation_baseline/formatted_replies.csv")

In [13]:
test_df = pd.read_csv("test_results.csv")

In [3]:
train_df.head()

,Utterance,Reply,Season,Episode,Dialogue_ID,Generated_Reply,bleu
0,also i was the point person on my companys tra...,you mustve had your hands full.,8,21,0,NaN,0.000000e+00
1,you mustve had your hands full.,that i did. that i did.,8,21,0,"He never took much risk.""Cain:!",1.154544e-231
2,that i did. that i did.,so lets talk a little bit about your duties.,8,21,0,rest at home with i drinkn u which i can!,9.475294e-232
3,so lets talk a little bit about your duties.,my duties? all right.,8,21,0,"First off, please consider: What kind of!",1.076601e-231
4,my duties? all right.,"now youll be heading a whole division, so youl...",8,21,0,"How did you find out about Markos, but with ex...",1.014495e-231


In [10]:
references = [s.encode('ascii', 'ignore').lower().decode('ascii').translate(str.maketrans('', '', string.punctuation)).split() 
              for s in list(train_df["Utterance"])]

In [22]:
candidates = [str(s).encode('ascii', 'ignore').lower().decode('ascii').translate(str.maketrans('', '', string.punctuation)).replace("unk", "").split() 
              for s in list(test_df["reply"])]

['its', 'extremely', 'complicated', 'listen', 'something']

In [26]:
sentence_bleu(references, candidates[0])

/home/scarecrow/anaconda2/envs/alexa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


7.600793306264725e-78

In [27]:
bleus = [sentence_bleu(references, sent) for sent in candidates]

/home/scarecrow/anaconda2/envs/alexa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/scarecrow/anaconda2/envs/alexa/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [31]:
test_bleus = [b for b in bleus if b > 0.000005]

In [34]:
sum(test_bleus) / len(test_bleus)

0.4488104525749111

### Perplexity

In [35]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

In [36]:
train_sentences = [' '.join(sent) for sent in references]
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in train_sentences]

In [38]:
n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, tokenized_text)
model = MLE(n)
model.fit(train_data, padded_vocab)

In [39]:
test_sentences = [' '.join(sent) for sent in candidates]
tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                for sent in test_sentences]

In [42]:
test_data, _ = padded_everygram_pipeline(n, tokenized_text)

In [43]:
perplexities = []

In [45]:
for test in test_data:
    try:
        perplexities.append(model.perplexity(test))
    except:
        pass

In [49]:
cp = [p for p in perplexities if p < 1000]

In [51]:
sum(cp) / len(cp)

366.78745135052594

In [52]:
cp.sort()

In [53]:
len(cp)

43

In [54]:
cp[21]

298.89395332623127